In [1]:
import datetime
import requests
import pandas as pd
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

file_handler = logging.FileHandler("logfile1.log")
formatter = logging.Formatter("%(asctime)s %(levelname)s:%(message)s")
file_handler.setFormatter(formatter)

logger.addHandler(file_handler)

start_timestamp = int(datetime.datetime(2022, 3, 3).timestamp())
end_timestamp = int(datetime.datetime.now().timestamp())

symbols = [
    'RUB=X', # USD/RUB (RUB=X)
    'GC=F', # Gold (GC=F)
    'CNY=X', # USD/CNY (CNY=X)
    'CL=F', # Crude Oil (CL=F)
    'ES=F', # E-Mini S&P 500 (ES=F)
]

all_data = pd.DataFrame()

for symbol in symbols:
    logging.info(f"Получение данных по: {symbol}")

    try:
        resp = requests.get(
            f'https://query1.finance.yahoo.com/v8/finance/chart/{symbol}',
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:45.0) Gecko/20100101 Firefox/45.0'},
            params = {'period1': start_timestamp, 'period2': end_timestamp, 'interval': '1d'},
        ).json()['chart']['result'][0]

        df = pd.DataFrame({
            'date': pd.to_datetime(resp['timestamp'], unit='s').normalize(), # .normalize для приведения временных меток к полуночи
            symbol: resp['indicators']['adjclose'][0]['adjclose']
        })

        logging.info(f"Данные {symbol} получены успешно.")

        if all_data.empty:
            all_data = df
        else:
            all_data = pd.merge(all_data, df, on='date', how='outer')

    except Exception as e:
        logging.error(f"Ошибка при получении данных по {symbol}: {e}")

all_data = all_data.sort_values('date').reset_index(drop=True)

all_data.to_csv("yahoo_finance_dataset.csv", index=False)


INFO:root:Получение данных по: RUB=X
INFO:root:Данные RUB=X получены успешно.
INFO:root:Получение данных по: GC=F
INFO:root:Данные GC=F получены успешно.
INFO:root:Получение данных по: CNY=X
INFO:root:Данные CNY=X получены успешно.
INFO:root:Получение данных по: CL=F
INFO:root:Данные CL=F получены успешно.
INFO:root:Получение данных по: ES=F
INFO:root:Данные ES=F получены успешно.
